In [1]:
import pandas as pd
import numpy as np
import joblib
import glob
import os
import sqlalchemy
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
def optimize_df_memory(df, verbose=False):
    """
    Downcasts numerical columns to smaller, more memory-efficient types.
    """
    initial_mem = df.memory_usage(deep=True).sum() / (1024**2)
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if 'float' in str(col_type):
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
        
        elif 'int' in str(col_type):
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)

        elif col_type == object:
            # Convert string columns to categorical type if they have low cardinality
            if df[col].nunique() < len(df) / 20:
                df[col] = df[col].astype('category')

    final_mem = df.memory_usage(deep=True).sum() / (1024**2)
    if verbose:
        print(f"Memory reduction: {initial_mem:.2f} MB -> {final_mem:.2f} MB ({100 * (initial_mem - final_mem) / initial_mem:.2f}% saved)")
    return df

In [3]:
# --- DATABASE CONNECTION & TABLE SETUP ---
DB_HOST = "10.192.5.43"
DB_NAME = "postgres"
DB_USER = "ml_db"
DB_PASS = "pass%401234"
DB_PORT = 5432
LOS_TABLE_NAME = "Ashirvad"
LOS_START_DATE = '2024-01-01'
LOS_END_DATE = '2024-12-31'

# --- GLOBAL CONSTANTS  ---
MAX_SEQUENCE_LENGTH = 8
ROLLING_WINDOW_SIZE = 3
TARGET_COL = 'NEXT_EMI_LABEL'
NEW_LMS_FOLDER_PATH = "TestData_Oct2025prediction"
GRACE_PERIOD_DAYS = 2 
PAID_PERCENTAGE_THRESHOLD = 0.90 
TWO_CLASS_STATUS_MAP = {0: 'Paid', 1: 'Not Paid'}

In [4]:
# --- VALIDATION MODE CONSTANT ---
TARGET_BACKTEST_EMI = 1  # 1 means we are predicting the last known EMI (EMI N) using data up to EMI N-1

# --- FEATURE LISTS (MUST BE EXACT MATCH TO TRAINING) ---
# FIX #1: Added RECENT_PAYMENT_SCORE to the sequential features list
SEQUENTIAL_COLS_NUMERICAL = [
    'INSTALLMENT_NO', 'INSTALLMENT_AMOUNT', 'DAYS_LATE', 'DAYS_BETWEEN_DUE_DATES',
    'PAID_RATIO', 'DELTA_DAYS_LATE', 'PAYMENT_SCORE', 'COMPOSITE_RISK', 
    'RECENT_PAYMENT_SCORE', 'PAYMENT_SCORE_RANK', 'IS_UNPAID', 'CURRENT_EMI_BEHAVIOR_LABEL'
]

STATIC_COLS_NUMERICAL = ['TOTAL_INCOME', 'TOTAL_EXPENSE', 'LOAN_AMOUNT', 'AGE', 'CYCLE']
STATIC_COLS_OHE = ['MARITAL_STATUS_NAME', 'STATE_NAME', 'LOAN_SCHEDULE_TYPE']
STATIC_EMBEDDING_COLS = ['OCCUPATION_NAME_ENCODED', 'LOAN_PURPOSE_ENCODED']

FLAG_ORDER = {'A': 4, 'B': 3, 'C': 2, 'D': 1}

In [5]:
# --- ARTIFACT LOADING ---
try:
    preprocessor = joblib.load('preprocessor.pkl')
    model = load_model('hybrid_lstm_model.h5', compile=False) 
    
    # FIX #2: Load the category mappings for embedding features
    try:
        category_mappings = joblib.load('embedding_category_mappings.pkl')
        print("✅ Category mappings loaded successfully.")
    except FileNotFoundError:
        print("⚠️ WARNING: embedding_category_mappings.pkl not found. Creating dummy mappings.")
        print("   This may cause incorrect predictions. Please save mappings from training.")
        category_mappings = {'occupation': {}, 'purpose': {}}
    
    print("✅ Preprocessor and Model loaded successfully.")
    
    # Extract the full list of numerical features that went into the scaler
    NUMERICAL_FEATURES_FINAL = SEQUENTIAL_COLS_NUMERICAL + STATIC_COLS_NUMERICAL
    
except FileNotFoundError as e:
    print(f"❌ ERROR: Could not find required artifact: {e}")
    exit()


✅ Category mappings loaded successfully.
✅ Preprocessor and Model loaded successfully.


In [ ]:


# ----------------------------------------------------------------------------------
# --- PART 1: DATA LOADING AND CORE FEATURE ENGINEERING (LOS & LMS) ---
# ----------------------------------------------------------------------------------

def load_data_from_db():
    """Fetches LOS data from PostgreSQL using SQLAlchemy and applies ALL preprocessing from training."""
    conn_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    try:
        engine = sqlalchemy.create_engine(conn_string)
        print(f"--- Loading LOS Data from PostgreSQL Table: {LOS_TABLE_NAME} ---")
        
        query = f"""
        SELECT *
        FROM "{LOS_TABLE_NAME}"
        WHERE "LOAN_DATE" BETWEEN '{LOS_START_DATE}' AND '{LOS_END_DATE}';
        """
        df = pd.read_sql_query(query, con=engine)
        
        # ===== FIX #3: ADD COMPLETE LOS PREPROCESSING (MATCHING TRAINING) =====
        
        print("--- Applying LOS Preprocessing (Matching Training Pipeline) ---")
        
        # 1. Convert PIN_CODE to string
        if 'PIN_CODE' in df.columns:
            df['PIN_CODE'] = df['PIN_CODE'].astype(str)
        
        # 2. Drop columns that were dropped in training
        columns_to_drop = [
            'CUSTOMER_ID', 'CUSTOMER_NAME', 'BRANCH_ID', 'TEMP_CUST_ID',
            'PHONE1', 'PHONE2', 'HOUSE_NAME', 'LOCALITY', 'STREET',
            'ALT_HOUSE_NAME', 'ALT_LOCALITY', 'ALT_STREET', 'CENTER_ID',
            'ALT_PIN_CODE', 'MARITAL_STATUS', 'LOAN_STATUS',
            'LOAN_STATUS_DESC', 'CLS_DT', 'CIBIL_ID', 'NPA_FLAG', 
            'NPA_FROM_DATE', 'NPA_TO_DATE', 'NPA_STATUS',
            'OCCUPATION_ID', 'APPLICATION_ID', 'TENURE_in_months', 
            'emi_paid', 'loan_paid_percentage', 'NPA_STATUS_UPDATED', 
            'NPA_STATUS_UPDATED_1', 'NPA_STATUS_UPDATED_2', 'YEAR'
        ]
        df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
        
        # 3. Drop CIBIL_SCORE and CUSTOMER_GRADING_SCORE
        df.drop(columns=['CIBIL_SCORE', 'CUSTOMER_GRADING_SCORE'], inplace=True, errors='ignore')
        
        # 4. Calculate AGE
        df['DATE_OF_BIRTH'] = pd.to_datetime(df['DATE_OF_BIRTH'], errors='coerce')
        df['LOAN_DATE'] = pd.to_datetime(df['LOAN_DATE'], errors='coerce')
        df['AGE'] = (df['LOAN_DATE'] - df['DATE_OF_BIRTH']).dt.days // 365.25
        df.drop(columns=['DATE_OF_BIRTH'], inplace=True)
        
        # 5. Filter CUSTOMER_FLAG (remove 'X' records)
        df = df[df['CUSTOMER_FLAG'] != 'X'].copy()
        
        # 6. Drop high-cardinality columns
        df.drop(columns=['BRANCH_NAME', 'CENTER_NAME'], errors='ignore', inplace=True)
        
        # 7. Impute missing values in numerical columns
        numerical_cols_to_impute = ['TOTAL_EXPENSE', 'TOTAL_INCOME', 'AGE']
        for col in numerical_cols_to_impute:
            if col in df.columns and df[col].isnull().sum() > 0:
                median_val = df[col].median()
                df[col] = df[col].fillna(median_val)
                print(f"   Imputed {df[col].isnull().sum()} missing values in {col} with median: {median_val:.2f}")
        
        # 8. Impute OCCUPATION_NAME with 'UNKNOWN'
        if 'OCCUPATION_NAME' in df.columns and df['OCCUPATION_NAME'].isnull().sum() > 0:
            df['OCCUPATION_NAME'] = df['OCCUPATION_NAME'].fillna('UNKNOWN')
            print(f"   Imputed missing OCCUPATION_NAME with 'UNKNOWN'")
        
        # 9. Consolidate MARITAL_STATUS_NAME
        if 'MARITAL_STATUS_NAME' in df.columns:
            df['MARITAL_STATUS_NAME'] = df['MARITAL_STATUS_NAME'].replace(
                ['UNMARRIED', 'SINGLE'], 'UNMARRIED/SINGLE')
            print(f"   Consolidated MARITAL_STATUS_NAME categories")
        
        print("✅ LOS Preprocessing Complete")
        
        # ===== END OF LOS PREPROCESSING =====
        
        df = optimize_df_memory(df, verbose=True)
        
        print(f"✅ Successfully fetched and preprocessed LOS data from DB. Shape: {df.shape}")
        return df
        
    except Exception as e:
        print(f"❌ Database connection or query failed: {e}")
        raise

def load_and_engineer_lms_data():
    """Loads, cleans, and engineers sequential features for LMS data."""
    all_lms_dfs = []
    lms_files = sorted(glob.glob(os.path.join(NEW_LMS_FOLDER_PATH, 'test_*.xlsx')))
    
    if not lms_files:
        raise FileNotFoundError(f"No Excel files found matching 'test_*.xlsx' in {NEW_LMS_FOLDER_PATH}.")
    
    print(f"--- Loading {len(lms_files)} LMS Excel file(s) ---")
    for file_name in lms_files:
        try:
            df = pd.read_excel(file_name)
            all_lms_dfs.append(df)
            print(f"   ✅ Loaded: {os.path.basename(file_name)}")
        except Exception as e:
            print(f"   ❌ Error loading {os.path.basename(file_name)}: {e}")
            
    df_lms = pd.concat(all_lms_dfs, ignore_index=True)
    df_lms = optimize_df_memory(df_lms, verbose=True)
    
    # --- LMS Core Cleaning ---
    print("--- Applying LMS Feature Engineering ---")
    df_lms['DUE_DATE'] = pd.to_datetime(df_lms['DUE_DATE'], errors='coerce')
    df_lms['PAID_DT'] = pd.to_datetime(df_lms['PAID_DT'], errors='coerce')
    df_lms.loc[df_lms['STATUS'] == 1, 'PAID_DT'] = pd.NaT 
    df_lms = df_lms.sort_values(by=['LOAN_ID', 'INSTALLMENT_NO']).reset_index(drop=True)
    
    # --- LMS Feature Engineering ---
    df_lms['DAYS_LATE'] = (df_lms['PAID_DT'] - df_lms['DUE_DATE']).dt.days
    df_lms['DAYS_BETWEEN_DUE_DATES'] = df_lms.groupby('LOAN_ID')['DUE_DATE'].diff().dt.days
    
    # Repayment Schedule Category
    df_lms['REPAYMENT_SCHEDULE_CAT'] = np.select(
        [
            df_lms['DAYS_BETWEEN_DUE_DATES'].isnull(), 
            df_lms['DAYS_BETWEEN_DUE_DATES'].isin([28, 29, 30, 31]), 
            df_lms['DAYS_BETWEEN_DUE_DATES'] == 7,
            df_lms['DAYS_BETWEEN_DUE_DATES'] == 14,
            df_lms['DAYS_BETWEEN_DUE_DATES'] == 56
        ],
        ['Initial', 'Monthly', 'Weekly', 'Bi-Weekly', 'Bi-Monthly'], 
        default='Other'
    )
    
    # Loan Schedule Type
    has_weekly = df_lms.groupby('LOAN_ID')['REPAYMENT_SCHEDULE_CAT'].transform(
        lambda x: 'Weekly' in x.unique())
    has_monthly = df_lms.groupby('LOAN_ID')['REPAYMENT_SCHEDULE_CAT'].transform(
        lambda x: 'Monthly' in x.unique())
    mode_schedule = df_lms.groupby('LOAN_ID')['REPAYMENT_SCHEDULE_CAT'].transform(
        lambda x: x.mode()[0] if not x.mode().empty else 'Initial')
    df_lms['LOAN_SCHEDULE_TYPE'] = np.where((has_weekly) & (has_monthly), 'Hybrid', mode_schedule)
    
    # Payment behavior features
    df_lms['IS_UNPAID'] = np.where(df_lms['STATUS'] == 1, 1, 0)
    df_lms['DAYS_LATE'] = df_lms['DAYS_LATE'].fillna(0)
    df_lms['PAID_RATIO'] = df_lms['PAID_AMOUNT'] / df_lms['INSTALLMENT_AMOUNT']
    df_lms['PAID_RATIO'] = df_lms['PAID_RATIO'].clip(upper=1.0)
    df_lms['DELTA_DAYS_LATE'] = df_lms.groupby('LOAN_ID')['DAYS_LATE'].diff().fillna(0)
    df_lms.loc[df_lms['IS_UNPAID'] == 1, 'DELTA_DAYS_LATE'] = 0 
    
    # Current EMI Behavior Label
    conditions_behavior = [
        (df_lms['STATUS'] == 1) | 
        ((df_lms['STATUS'] == 0) & (df_lms['DAYS_LATE'] > GRACE_PERIOD_DAYS)) | 
        ((df_lms['STATUS'] == 2) & (
            (df_lms['DAYS_LATE'] > GRACE_PERIOD_DAYS) | 
            (df_lms['PAID_AMOUNT'] / df_lms['INSTALLMENT_AMOUNT'] < PAID_PERCENTAGE_THRESHOLD)
        )),
        (df_lms['STATUS'] == 0) & (df_lms['DAYS_LATE'] <= GRACE_PERIOD_DAYS),
        (df_lms['STATUS'] == 2) & (df_lms['DAYS_LATE'] <= GRACE_PERIOD_DAYS) & 
        (df_lms['PAID_AMOUNT'] / df_lms['INSTALLMENT_AMOUNT'] >= PAID_PERCENTAGE_THRESHOLD)
    ]
    choices_behavior = [1, 0, 0]
    df_lms['CURRENT_EMI_BEHAVIOR_LABEL'] = np.select(
        conditions_behavior, choices_behavior, default=-1).astype(np.int8)
    
    # Composite Risk and Payment Score
    df_lms['REMAINING_EMI_RATIO'] = 1 - df_lms["PAID_RATIO"]
    df_lms['COMPOSITE_RISK'] = df_lms['DAYS_LATE'] + (df_lms['REMAINING_EMI_RATIO'] * 10)
    
    conditions_score = [
        df_lms['IS_UNPAID'] == 1,
        (df_lms['CURRENT_EMI_BEHAVIOR_LABEL'] == 1) & (df_lms['IS_UNPAID'] == 0),
        (df_lms['CURRENT_EMI_BEHAVIOR_LABEL'] == 0) & (df_lms['DAYS_LATE'] > 0) & 
        (df_lms['DAYS_LATE'] <= GRACE_PERIOD_DAYS) & (df_lms['IS_UNPAID'] == 0),
        (df_lms['CURRENT_EMI_BEHAVIOR_LABEL'] == 0) & (df_lms['DAYS_LATE'] <= 0) & 
        (df_lms['IS_UNPAID'] == 0)
    ]
    
    choices_score = [
        -100,
        np.maximum(0.0, np.minimum(0.30, 0.30 - (df_lms['COMPOSITE_RISK'] * 0.03))),
        1.0 / (1 + df_lms['COMPOSITE_RISK']),
        1.5 + (np.abs(df_lms['DAYS_LATE']) / 10)
    ]
    df_lms['PAYMENT_SCORE'] = np.select(conditions_score, choices_score, default=0).astype(np.float32)
    
    rank_choices = [4, 3, 2, 1]
    df_lms['PAYMENT_SCORE_RANK'] = np.select(conditions_score, rank_choices).astype(np.int8)

    # --- VALIDATION MODE: DEFINE TARGET (EMI N) ---
    actual_targets = df_lms.groupby('LOAN_ID').tail(TARGET_BACKTEST_EMI).copy()
    actual_targets = actual_targets[['LOAN_ID', 'CURRENT_EMI_BEHAVIOR_LABEL']].rename(
        columns={'CURRENT_EMI_BEHAVIOR_LABEL': 'ACTUAL_NEXT_EMI_ISSUE'}
    )
    
    print(f"✅ LMS Feature Engineering complete. Shape: {df_lms.shape}")
    return df_lms, actual_targets

def merge_and_finalize_data():
    """Merges LOS and LMS, applies final feature engineering, and performs back-test trimming."""
    
    # 1. Load Data
    df_los = load_data_from_db()
    df_lms_full, df_actual_targets = load_and_engineer_lms_data()
    
    # 2. VALIDATION MODE: Trim the last EMI (EMI N) from the sequence
    print(f"\n--- Back-testing Mode: Trimming last {TARGET_BACKTEST_EMI} EMI(s) ---")
    indices_to_drop = df_lms_full.groupby('LOAN_ID').tail(TARGET_BACKTEST_EMI).index
    df_lms = df_lms_full.drop(indices_to_drop).reset_index(drop=True)
    print(f"   LMS records after trimming: {df_lms.shape[0]}")
    
    # 3. Rename and Merge LOS
    RENAME_MAP_LOS = {
        'LOAN_AMOUNT': 'LOAN_AMOUNT_STATIC', 
        'TENURE': 'TENURE_STATIC', 
        'INTEREST_RATE': 'INTEREST_RATE_STATIC'
    }
    df_los.rename(columns=RENAME_MAP_LOS, inplace=True)
    df_combined = pd.merge(df_los, df_lms, on='LOAN_ID', how='inner', suffixes=('_static', '_lms'))
    print(f"   Combined shape after merge: {df_combined.shape}")
    
    # 4. Final Cleaning and Rename
    df_combined.rename(columns={
        "LOAN_DATE_static": "LOAN_DATE", 
        "LOAN_AMOUNT_STATIC": "LOAN_AMOUNT", 
        "TENURE_STATIC": "TENURE", 
        "INTEREST_RATE_STATIC": "INTEREST_RATE"
    }, inplace=True)
    
    cols_to_drop = [
        c for c in ['LOAN_DATE_lms', 'LOAN_AMOUNT_lms', 'TENURE_lms', 'INTEREST_RATE_lms', 
                    'DISBURSED_AMOUNT', 'PIN_CODE', 'IS_DAYS_LATE_MISSING', 'STATUS', 
                    'PAID_DT', 'PAID_AMOUNT', 'REMAINING_EMI_RATIO', 'TENURE', 'INTEREST_RATE'] 
        if c in df_combined.columns
    ]
    df_combined.drop(columns=cols_to_drop, errors='ignore', inplace=True)
    df_combined["DAYS_BETWEEN_DUE_DATES"].fillna(0, inplace=True)

    # 5. Rolling Feature (RECENT_PAYMENT_SCORE) - Shifted, then imputed
    print("   Creating RECENT_PAYMENT_SCORE (rolling window feature)...")
    df_combined['RECENT_PAYMENT_SCORE'] = df_combined.groupby('LOAN_ID')['PAYMENT_SCORE'].rolling(
        window=ROLLING_WINDOW_SIZE, min_periods=1
    ).mean().reset_index(level=0, drop=True).astype(np.float32)
    df_combined['RECENT_PAYMENT_SCORE'] = df_combined.groupby('LOAN_ID')['RECENT_PAYMENT_SCORE'].shift(1)
    overall_mean_score = df_combined['PAYMENT_SCORE'].mean()
    df_combined['RECENT_PAYMENT_SCORE'].fillna(overall_mean_score, inplace=True)

    # 6. Custom Encoding
    print("   Applying custom encoding for CUSTOMER_FLAG, OCCUPATION_NAME, LOAN_PURPOSE...")
    df_combined['CUSTOMER_FLAG_ENCODED'] = df_combined['CUSTOMER_FLAG'].astype(str).map(FLAG_ORDER).fillna(0).astype(np.int8)
    
    # FIX #4: Use saved category mappings for consistent encoding
    if category_mappings['occupation'] and category_mappings['purpose']:
        # Create reverse mappings (category -> code)
        occupation_to_code = {v: k+1 for k, v in category_mappings['occupation'].items()}
        purpose_to_code = {v: k+1 for k, v in category_mappings['purpose'].items()}
        
        # Apply encoding with training mappings, use 0 for unknown categories
        df_combined['OCCUPATION_NAME_ENCODED'] = df_combined['OCCUPATION_NAME'].astype(str).map(
            occupation_to_code).fillna(0).astype(np.int16)
        df_combined['LOAN_PURPOSE_ENCODED'] = df_combined['LOAN_PURPOSE'].astype(str).map(
            purpose_to_code).fillna(0).astype(np.int16)
        
        print(f"      OCCUPATION_NAME: {df_combined['OCCUPATION_NAME_ENCODED'].nunique()} unique codes")
        print(f"      LOAN_PURPOSE: {df_combined['LOAN_PURPOSE_ENCODED'].nunique()} unique codes")
    else:
        # Fallback to categorical codes (may be inconsistent with training)
        print("   ⚠️ WARNING: Using fallback encoding (may be inconsistent with training)")
        df_combined['OCCUPATION_NAME_ENCODED'] = df_combined['OCCUPATION_NAME'].astype(
            'category').cat.codes + 1
        df_combined['LOAN_PURPOSE_ENCODED'] = df_combined['LOAN_PURPOSE'].astype(
            'category').cat.codes + 1
    
    df_combined.drop(columns=['CUSTOMER_FLAG', 'OCCUPATION_NAME', 'LOAN_PURPOSE', 'LOAN_DATE'], 
                     errors='ignore', inplace=True)
    
    # 7. OHE Sequential Categorical Feature
    print("   One-hot encoding REPAYMENT_SCHEDULE_CAT...")
    df_combined = pd.get_dummies(df_combined, columns=['REPAYMENT_SCHEDULE_CAT'], prefix='REPAYMENT_CAT')

    # 8. Filter for the latest sequence for prediction (up to EMI N-1)
    print("   Filtering to keep only necessary sequence (last MAX_SEQUENCE_LENGTH EMIs)...")
    df_combined['SEQUENCE_COUNT'] = df_combined.groupby('LOAN_ID').cumcount() + 1
    df_combined['REVERSE_SEQUENCE_COUNT'] = df_combined.groupby('LOAN_ID')['SEQUENCE_COUNT'].transform(
        'max') - df_combined['SEQUENCE_COUNT']
    df_combined_filtered = df_combined[df_combined['REVERSE_SEQUENCE_COUNT'] < MAX_SEQUENCE_LENGTH].copy()
    df_combined_filtered.drop(columns=['SEQUENCE_COUNT', 'REVERSE_SEQUENCE_COUNT'], inplace=True)
    
    # 9. Merge Actual Target for Validation
    df_combined_filtered = pd.merge(df_combined_filtered, df_actual_targets, on='LOAN_ID', how='left')
    df_combined_filtered['ACTUAL_NEXT_EMI_ISSUE'] = df_combined_filtered.groupby(
        'LOAN_ID')['ACTUAL_NEXT_EMI_ISSUE'].transform('max')
    df_combined_filtered.dropna(subset=['ACTUAL_NEXT_EMI_ISSUE'], inplace=True)

    print(f"✅ Data preparation complete. Final shape: {df_combined_filtered.shape}")
    print(f"   Unique loans: {df_combined_filtered['LOAN_ID'].nunique()}")
    
    return df_combined_filtered

def preprocess_and_predict(df_full):
    """Applies preprocessor, reshapes sequences, and runs inference."""
    
    print("\n--- Step: Preprocessing and Prediction ---")
    
    # FIX #5: Alignment of OHE Columns with Training
    training_ohe_cols = [c for c in preprocessor.feature_names_in_ if c.startswith('REPAYMENT_CAT_')]
    print(f"   Aligning OHE columns ({len(training_ohe_cols)} expected from training)...")
    for col in training_ohe_cols:
        if col not in df_full.columns:
            df_full[col] = 0
            print(f"      Added missing column: {col}")
    
    # FIX #6: Get expected features from preprocessor and ensure all are present
    expected_features = list(preprocessor.feature_names_in_)
    
    # Add CUSTOMER_FLAG_ENCODED to static numerical if not present
    if 'CUSTOMER_FLAG_ENCODED' not in STATIC_COLS_NUMERICAL:
        STATIC_COLS_NUMERICAL_FINAL = STATIC_COLS_NUMERICAL + ['CUSTOMER_FLAG_ENCODED']
    else:
        STATIC_COLS_NUMERICAL_FINAL = STATIC_COLS_NUMERICAL
    
    # Define the columns that should be transformed (numerical + OHE categorical)
    NUMERICAL_FEATURES_WITH_OHE = SEQUENTIAL_COLS_NUMERICAL + STATIC_COLS_NUMERICAL_FINAL + training_ohe_cols
    ALL_TRANSFORM_COLS = NUMERICAL_FEATURES_WITH_OHE + STATIC_COLS_OHE
    
    # Check for missing columns and add them with zeros
    missing_cols = set(expected_features) - set(df_full.columns)
    if missing_cols:
        print(f"   ⚠️ Adding {len(missing_cols)} missing columns with zeros")
        for col in missing_cols:
            df_full[col] = 0
    
    # Ensure column order matches training
    df_for_transform = df_full[expected_features].copy()
    
    # FIX #7: Apply ColumnTransformer
    print("   Applying StandardScaler and OneHotEncoder...")
    X_scaled_ohe = preprocessor.transform(df_for_transform)
    ALL_FINAL_COLS = list(preprocessor.get_feature_names_out())
    
    # Reconstruct DataFrame
    X_predict_df = pd.DataFrame(X_scaled_ohe, columns=ALL_FINAL_COLS, index=df_full.index)
    
    # Add back non-transformed columns
    X_predict_df['LOAN_ID'] = df_full['LOAN_ID'].values
    X_predict_df['ACTUAL_NEXT_EMI_ISSUE'] = df_full['ACTUAL_NEXT_EMI_ISSUE'].values
    for col in STATIC_EMBEDDING_COLS:
        X_predict_df[col] = df_full[col].values

    # Define Final Input Feature Sets with correct prefixes
    LSTM_INPUT_COLS_FINAL = [f"num__{c}" for c in SEQUENTIAL_COLS_NUMERICAL] + \
                            [f"num__{c}" for c in training_ohe_cols]
    
    STATIC_DENSE_COLS_FINAL = [f"num__{c}" for c in STATIC_COLS_NUMERICAL_FINAL] + \
                              [c for c in ALL_FINAL_COLS if c.startswith('cat__')]

    print(f"   LSTM input features: {len(LSTM_INPUT_COLS_FINAL)}")
    print(f"   Static dense features: {len(STATIC_DENSE_COLS_FINAL)}")
    print(f"   Embedding features: {len(STATIC_EMBEDDING_COLS)}")

    # Sequence Reshaping Function
    def reshape_for_prediction(X_df, lstm_cols, static_dense_cols, embedding_cols, max_len):
        grouped = X_df.groupby('LOAN_ID')
        loan_ids = list(grouped.groups.keys())
        
        X_lstm = np.zeros((len(loan_ids), max_len, len(lstm_cols)), dtype=np.float32)
        X_static_dense = np.zeros((len(loan_ids), len(static_dense_cols)), dtype=np.float32)
        X_static_embed = np.zeros((len(loan_ids), len(embedding_cols)), dtype=np.int16)
        y_actual = np.zeros((len(loan_ids),), dtype=np.int8)
        
        for i, loan_id in enumerate(loan_ids):
            loan_data = grouped.get_group(loan_id)
            sequence = loan_data[lstm_cols].values
            
            # Pad or truncate sequence
            if len(sequence) >= max_len:
                X_lstm[i, :, :] = sequence[-max_len:]
            else:
                X_lstm[i, -len(sequence):, :] = sequence

            # Get static features from last record
            last_record = loan_data.iloc[-1]
            X_static_dense[i, :] = last_record[static_dense_cols].values
            X_static_embed[i, :] = last_record[embedding_cols].values.astype(np.int16)
            y_actual[i] = last_record['ACTUAL_NEXT_EMI_ISSUE'].astype(np.int8)

        return X_lstm, X_static_dense, X_static_embed, loan_ids, y_actual

    # Execute reshaping
    print("   Reshaping sequences for model input...")
    X_predict_lstm, X_predict_static_dense, X_predict_static_embed, prediction_loan_ids, y_actual = \
        reshape_for_prediction(X_predict_df, LSTM_INPUT_COLS_FINAL, STATIC_DENSE_COLS_FINAL, 
                              STATIC_EMBEDDING_COLS, MAX_SEQUENCE_LENGTH)

    print(f"   Reshaped data shapes:")
    print(f"      LSTM: {X_predict_lstm.shape}")
    print(f"      Static Dense: {X_predict_static_dense.shape}")
    print(f"      Embedding: {X_predict_static_embed.shape}")

    # Model Inference
    print("\n   Running model inference...")
    prediction_inputs = {
        'lstm_input': X_predict_lstm, 
        'static_dense_input': X_predict_static_dense, 
        'embedding_input': X_predict_static_embed
    }

    y_pred_proba = model.predict(prediction_inputs, verbose=0)[:, 0]
    y_pred_class = (y_pred_proba > 0.5).astype(np.int8)

    # Inverse Transform Installment Number
    print("   Extracting and inverse-transforming installment numbers...")
    INSTALLMENT_NO_COL_NAME_SCALED = 'num__INSTALLMENT_NO'
    CURRENT_EMI_BEHAVIOR_LABEL_COL_NAME_SCALED = 'num__CURRENT_EMI_BEHAVIOR_LABEL'
    
    try:
        INSTALLMENT_NO_IDX = LSTM_INPUT_COLS_FINAL.index(INSTALLMENT_NO_COL_NAME_SCALED)
        CURRENT_EMI_BEHAVIOR_LABEL_IDX = LSTM_INPUT_COLS_FINAL.index(CURRENT_EMI_BEHAVIOR_LABEL_COL_NAME_SCALED)
    except ValueError as e:
        print(f"   ❌ ERROR: Required column not found in LSTM features: {e}")
        raise
    
    LAST_TIME_STEP_IDX = X_predict_lstm.shape[1] - 1
    
    # Get scaler parameters for inverse transform
    scaler = preprocessor.named_transformers_['num']
    try:
        original_feature_index = list(preprocessor.feature_names_in_).index('INSTALLMENT_NO')
    except ValueError:
        print("   ⚠️ WARNING: INSTALLMENT_NO not found in preprocessor features. Using index 0.")
        original_feature_index = 0
    
    install_no_mean = scaler.mean_[original_feature_index]
    install_no_scale = scaler.scale_[original_feature_index]

    # Inverse transform
    scaled_install_no = X_predict_lstm[:, LAST_TIME_STEP_IDX, INSTALLMENT_NO_IDX]
    unscaled_install_no = (scaled_install_no * install_no_scale) + install_no_mean
    install_no_final_unscaled = np.round(unscaled_install_no + 1).astype(np.int16)

    # Extract Current Behavior
    behavior_label_final = X_predict_lstm[:, LAST_TIME_STEP_IDX, CURRENT_EMI_BEHAVIOR_LABEL_IDX].astype(np.int8)

    # Final Output DataFrame
    results_df = pd.DataFrame({
        'LOAN_ID': prediction_loan_ids,
        'LAST_EMI_USED_IN_PREDICTION': np.round(unscaled_install_no).astype(np.int16),
        'PREDICTED_EMI_NO': install_no_final_unscaled,
        'LAST_EMI_BEHAVIOR_USED_DESC': [TWO_CLASS_STATUS_MAP.get(lbl, 'Unknown') for lbl in behavior_label_final],
        'PROBABILITY_OF_EMI_ISSUE': y_pred_proba.astype(np.float32),
        'PREDICTED_EMI_ISSUE': y_pred_class,
        'PREDICTED_EMI_STATUS': [TWO_CLASS_STATUS_MAP.get(pred, 'Unknown') for pred in y_pred_class],
        'ACTUAL_EMI_ISSUE': y_actual,
        'ACTUAL_EMI_STATUS': [TWO_CLASS_STATUS_MAP.get(actual, 'Unknown') for actual in y_actual]
    })
    
    return results_df, y_pred_class, y_actual

# --- MAIN EXECUTION BLOCK ---
if __name__ == "__main__":
    try:
        print("="*80)
        print("HYBRID LSTM MODEL - BACK-TESTING PIPELINE")
        print("="*80)
        
        # Load, Merge, and Engineer Data
        df_full_data = merge_and_finalize_data()

        # Preprocess, Reshape, and Predict
        final_predictions_df, y_pred, y_actual = preprocess_and_predict(df_full_data)

        # --- Performance Calculation ---
        print("\n--- Calculating Performance Metrics ---")
        accuracy = accuracy_score(y_actual, y_pred)
        cm = confusion_matrix(y_actual, y_pred)
        report = classification_report(y_actual, y_pred, 
                                      target_names=['Paid (0)', 'Not Paid (1)'], 
                                      output_dict=True)

        # Export Results
        OUTPUT_FILE = 'Oct2025_Backtest_Predictions_Results.xlsx'
        final_predictions_df.to_excel(OUTPUT_FILE, index=False)
        
        print("\n" + "="*80)
        print("✅ BACK-TESTING COMPLETE!")
        print("="*80)
        print(f"Total Loans Evaluated: {final_predictions_df.shape[0]}")
        print(f"Output saved to: {OUTPUT_FILE}")

        print("\n### 📈 MODEL PERFORMANCE ON BACK-TEST DATA ###")
        print(f"Overall Accuracy: {accuracy:.4f}\n")
        
        print("Confusion Matrix:")
        cm_df = pd.DataFrame(cm, 
                            index=['Actual Paid (0)', 'Actual Not Paid (1)'], 
                            columns=['Predicted Paid (0)', 'Predicted Not Paid (1)'])
        print(cm_df)
        
        print("\n Classification Report (Key Metrics):")
        print(f"Precision (Not Paid): {report['Not Paid (1)']['precision']:.4f}")
        print(f"Recall (Not Paid): {report['Not Paid (1)']['recall']:.4f}")
        print(f"F1-Score (Not Paid): {report['Not Paid (1)']['f1-score']:.4f}")

        print("\n" + "="*80)
        print("\nSample Results (First 10 Predictions):")
        print("="*80)
        display_cols = ['LOAN_ID', 'PREDICTED_EMI_NO', 'PROBABILITY_OF_EMI_ISSUE', 
                       'PREDICTED_EMI_STATUS', 'ACTUAL_EMI_STATUS']
        print(final_predictions_df[display_cols].head(10).to_string(index=False))
        
        print("\n✅ All operations completed successfully!")

    except Exception as e:
        print("\n" + "="*80)
        print("❌ FATAL ERROR IN PREDICTION SCRIPT")
        print("="*80)
        print(f"Error Details: {e}")
        import traceback
        print("\nFull Traceback:")
        print(traceback.format_exc())

HYBRID LSTM MODEL - BACK-TESTING PIPELINE
--- Loading LOS Data from PostgreSQL Table: Ashirvad ---
--- Applying LOS Preprocessing (Matching Training Pipeline) ---
   Imputed 0 missing values in TOTAL_EXPENSE with median: 1800.00
   Imputed 0 missing values in TOTAL_INCOME with median: 21000.00
   Imputed missing OCCUPATION_NAME with 'UNKNOWN'
   Consolidated MARITAL_STATUS_NAME categories
✅ LOS Preprocessing Complete
Memory reduction: 571.55 MB -> 134.06 MB (76.54% saved)
✅ Successfully fetched and preprocessed LOS data from DB. Shape: (1095563, 16)
--- Loading 24 LMS Excel file(s) ---
   ✅ Loaded: test_001.xlsx
   ✅ Loaded: test_002.xlsx
   ✅ Loaded: test_003.xlsx
   ✅ Loaded: test_004.xlsx
   ✅ Loaded: test_005.xlsx
   ✅ Loaded: test_006.xlsx
   ✅ Loaded: test_007.xlsx
   ✅ Loaded: test_008.xlsx
   ✅ Loaded: test_009.xlsx
   ✅ Loaded: test_010.xlsx
   ✅ Loaded: test_011.xlsx
   ✅ Loaded: test_012.xlsx
   ✅ Loaded: test_013.xlsx
   ✅ Loaded: test_014.xlsx
   ✅ Loaded: test_015.xlsx
  